In [16]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/train_synthetics.csv')

# Filter rows where '6_way_label' equals 0 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows_SD = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('SD'))]
filtered_rows_DL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('DL'))]
filtered_rows_GL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('GL'))]

# Concatenate columns for all groups
result = pd.concat([filtered_rows_SD, filtered_rows_DL, filtered_rows_GL])
print(len(result))

# Save the result to a new CSV file
result.to_csv('csv/train_class_0_already_generated.csv', index=False)

print("Filtered IDs saved to train_class_0_already_generated.csv")

55610
Filtered IDs saved to train_class_0_already_generated.csv


------------------

TRAIN CLASS 0: collect the 55k fakes already generated, and save a csv for the 107k pristine and the 107k-55k to be generated

In [17]:
import pandas as pd

MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_0 = "csv/train_class_0_already_generated.csv"

train = pd.read_csv(MULTIMODAL_TRAIN_CLEANED_WITH_CLASS_TSV, sep='\t')
train_filtered = train[train['6_way_label'] == 0]

train_class_0_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_0)
print("len of train_class_0_already_generated: "+str(len(train_class_0_already_generated)))

# Filter rows from train DataFrame that are not in the 'id' column of the other DataFrame
train_not_in_other = train_filtered[~train_filtered['id'].isin(train_class_0_already_generated['id'])]

# Select the first 107,549 rows from train_not_in_other
train_pristine = train_not_in_other.head(107548)

# Save train_pristine to CSV
print("len of train_pristine: "+str(len(train_pristine)))
train_pristine.to_csv('csv/train_pristine.csv', index=False)

# Filter rows from train_filtered that are not in train_class_0_already_generated or train_pristine
train_remaining = train_filtered[~train_filtered['id'].isin(train_class_0_already_generated['id']) & ~train_filtered['id'].isin(train_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of train_class_0_to_be_generated: "+str(len(train_remaining)))
train_remaining.to_csv('csv/train_class_0_to_be_generated.csv', index=False)
print("train_class_0_already_generated: 55.610 + train_class_0_to_be_generated: 51.938 = " + str(55610+51938))

len of train_class_0_already_generated: 55610
len of train_pristine: 107548
len of train_class_0_to_be_generated: 51939
train_class_0_already_generated: 55.610 + train_class_0_to_be_generated: 51.938 = 107548


rename the columns real_image and fake_image to pristine_image and generated_image inside train_class_0_already_generated.csv

In [20]:
df = pd.read_csv('csv/train_class_0_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/train_class_0_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in train_class_0_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [21]:
df = pd.read_csv('csv/train_class_0_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train_class_0_already_generated.csv", index=False)

reduce columns of train_class_0_to_be_generated and train_pristine csv:

In [22]:
import pandas as pd

df = pd.read_csv('csv/train_class_0_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/train_class_0_to_be_generated.csv", index=False)

In [23]:
import pandas as pd

df = pd.read_csv('csv/train_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/train_pristine.csv", index=False)

check if every id inside the 250k train_tsv_with_class.csv is contained in the union of the 3 csv train_pristine + train_class_0_already_generated + train_class_0_to_be_generated

In [35]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/train_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/train_class_0_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/train_class_0_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t')['6_way_label'] == 0, 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 0.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 0.")

215097
215097
The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 0.


Now i print out how many StableDiffusion, Dreamlike and Glide images i have already generated:

In [26]:
df = pd.read_csv("csv/train_class_0_already_generated.csv")

# Count the rows where the 'fake_id' column starts with 'SD', 'DL', or 'GL'
count_SD = len(df[df['fake_id'].str.startswith('SD')])
count_DL = len(df[df['fake_id'].str.startswith('DL')])
count_GL = len(df[df['fake_id'].str.startswith('GL')])

print("Number of rows where 'fake_id' starts with 'SD':", count_SD)
print("Number of rows where 'fake_id' starts with 'DL':", count_DL)
print("Number of rows where 'fake_id' starts with 'GL':", count_GL)

Number of rows where 'fake_id' starts with 'SD': 22350
Number of rows where 'fake_id' starts with 'DL': 22044
Number of rows where 'fake_id' starts with 'GL': 11216


Thus, to get to 107.549 generated images i want to generate other:

- SD: 40.000 - 22.350 = 17.650
- DL: 40.000 - 22.044 = 17.956
- GL: (107.549 - 80.000) - 11.216 = 16.333

so 51.939 in total from the train_class_0_to_be_generated.csv, but first i need to caption all the images from train_class_0_to_be_generated.csv

In [36]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train_class_0_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "C:/Users/nello/Desktop/TESI/dataset_after_merging_WITH_DUPLICATES"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 51939
